In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
import math
from math import log

from  pprint import pformat
from collections import namedtuple

In [4]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
           ['青年', '否', '否', '好', '否'],
           ['青年', '是', '否', '好', '是'],
           ['青年', '是', '是', '一般', '是'],
           ['青年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '好', '否'],
           ['中年', '是', '是', '好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '好', '是'],
           ['老年', '是', '否', '好', '是'],
           ['老年', '是', '否', '非常好', '是'],
           ['老年', '否', '否', '一般', '否'],
           ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    
    return datasets,labels

In [5]:
datasets,labels=create_data()
train_data=pd.DataFrame(datasets,columns=labels)
train_data2=train_data
train_data2.columns=['Age','Work','Hourse','Loan','Category']

In [155]:
train_data2

,Age,Work,Hourse,Loan,Category
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [12]:
#熵
def emp_ent(label):
    length=len(label)
    s=0
    for k ,v in dict(label.value_counts()).items():
        p=v/length
        s+=p * math.log(p,2)
    return -s

In [13]:
emp_ent(train_data2.Category)

0.9709505944546686

In [16]:
# 条件熵
def cond_ent(data,col,label):
    length=len(data[col])
    s=0
    for k,v in dict(data[col].value_counts()).items():
        p=v/length
        s+=p*emp_ent(data[data[col]==k][label])
    return s
    
    
    

In [17]:
cond_ent(train_data2,'Age','Category')

0.8879430945988998

In [22]:
def gain_func(data,col,label):
    return emp_ent(data[label]) - cond_ent(data,col,label)
    

In [23]:
gain_func(train_data2,'Age','Category')

0.08300749985576883

In [26]:
def max_gain(data,label):
    ret={}
    for col in data.columns:
        if col != label:
            ret[col]=gain_func(data,col,label)
    print(ret)
    return sorted(ret.items(),key=lambda x:x[1])[-1]

            

In [27]:
max_gain(train_data2,'Category')

{'Age': 0.08300749985576883, 'Work': 0.32365019815155627, 'Hourse': 0.4199730940219749, 'Loan': 0.36298956253708536}


('Hourse', 0.4199730940219749)

In [28]:
def gain_rate(data,col,label):
    return gain_func(data,col,label) / emp_ent(data[label])

In [29]:
gain_rate(train_data2,'Age','Category')

0.08549096146585063

In [32]:
def max_gain_rate(data,label):
    ret={}
    for col in data.columns:
        if col != label:
            ret[col]=gain_rate(data,col,label)
    print(ret)
    return sorted(ret.items(),key=lambda x:x[1])[-1]

In [33]:
max_gain_rate(train_data2,'Category')

{'Age': 0.08549096146585063, 'Work': 0.3333333333333334, 'Hourse': 0.4325380677663126, 'Loan': 0.3738496733100589}


('Hourse', 0.4325380677663126)

In [34]:
from collections import namedtuple
from pprint import pformat

In [35]:
class dttree(namedtuple('dttree','feat,value,label,subtree')):
    def __repr__(self):
        return pformat(tuple(self))

In [85]:
def create_dttree(data,label):
    col= data.columns.values.tolist()
#     col.remove(label)
    
    if data is None:
        return dttree(None,None,None,None)
    
    
    if len(data[label].unique())==1:
        return dttree(None,None,data[label][0],None)
    
    if data.shape[1]==1 and (label !='' or label is not None):
        label=sorted(dict(data[label].value_counts()).items(),key=lambda x:x[1])[-1][0]
        return dttree(None,None,label,None)
    
#     if len(col)==2:
#         return dttree(col[0],None,
#                       sorted(dict(data[label].value_counts()).items())[-1][0],None)
    
#     print(data)
    fn,gv=max_gain_rate(data,label)
    ts=[]
    tl=None
    tv=None
#     print(fn)
#     print(col)
#     col.remove(fn)
    for v in list(set(data[fn])):
        lvl =data[data[fn]==v][label].unique().tolist()
        if len(lvl)==1:
            ts.append(dttree(None,v,lvl[0],None))
        
        else :
#             print(col)
            col.remove(fn)
#             col.append(label)

            temp=create_dttree(data[data[fn]==v][col],label)
            ts.append(dttree(None,v,None,temp))
    
    return dttree(fn,None,None,ts)
            

In [86]:
create_dttree(train_data2,'Category')

{'Age': 0.08549096146585063, 'Work': 0.3333333333333334, 'Hourse': 0.4325380677663126, 'Loan': 0.3738496733100589}
{'Age': 0.274017542121281, 'Work': 1.0, 'Loan': 0.5160116947475206}


('Hourse',
 None,
 None,
 [(None,
 '否',
 None,
 ('Work', None, None, [(None, '否', '否', None), (None, '是', '是', None)])),
  (None, '是', '是', None)])

In [131]:
def kdtree_predict(tr,x,last_col=None):
#     print(tr)
    tn=tr.feat
    if tn is  None:
        tn=last_col
    tv=tr.value
    tl=tr.label
#     print(tn)
#     print(x[tn])
#     print(x[tn][0])
    test_v=x[tn][0]

    
    if test_v==tv and tl is not None:
        l=tl
        return l
    
    if test_v==tv :
        l=kdtree_predict(tr.subtree,x,tn)
        return l
            
    
    if tv is None:
        i=0
        while tr.subtree[i].value!= test_v:
            i+=1
        l= kdtree_predict(tr.subtree[i],x,tn)
    return l
    

In [134]:
t_data=np.array(['老年', '是', '是', '一般'])
t_data=t_data.reshape(1,4)
t_col=train_data2.columns.values.tolist()[:-1]
# test_data=pd.DataFrame(np.array(t_data),columns=t_col)
test_data=pd.DataFrame(t_data,columns=t_col)
test_data

,Age,Work,Hourse,Loan
0,老年,是,是,一般


In [135]:
dt=create_dttree(train_data2,'Category')
kdtree_predict(dt,test_data)

{'Age': 0.08549096146585063, 'Work': 0.3333333333333334, 'Hourse': 0.4325380677663126, 'Loan': 0.3738496733100589}
{'Age': 0.274017542121281, 'Work': 1.0, 'Loan': 0.5160116947475206}


'是'

In [129]:
test_data['Hourse'][0]

'否'

In [150]:
def gini_func(data,col,label,lv='是'):
    length1=len(data[col])
    ret={}
    for k,v in dict(data[col].value_counts()).items():
        p1=v/length1
#         print(p1)
        p2=data[(data[col]==k) & (data[label]==lv)].shape[0] /len(data[data[col]==k][label]) 
#         print(p2)
        p3=data[(data[col]!=k) & (data[label]==lv)].shape[0] /len(data[data[col]!=k][label])
#         print(p3)
        ret[k]=p1*(2*p2*(1-p2)) + (1-p1)*(2*p3*(1-p3))
#         print(ret)
    return ret
    

In [151]:
gini_func(train_data2,'Age','Category')

{'中年': 0.48, '老年': 0.44, '青年': 0.44000000000000006}

In [154]:
for c in train_data2.columns:
    if c !='Category':
        print(c+'->'+str(gini_func(train_data2,c,'Category')))

Age->{'老年': 0.44, '青年': 0.44000000000000006, '中年': 0.48}
Work->{'否': 0.31999999999999995, '是': 0.32}
Hourse->{'否': 0.26666666666666666, '是': 0.26666666666666666}
Loan->{'好': 0.4740740740740741, '一般': 0.32, '非常好': 0.36363636363636365}


In [164]:
a=sorted(gini_func(train_data2,'Age','Category').items(),key=lambda x:x[1])
# len(a)
a[-1][0]

'中年'

In [204]:
def min_gini(data,label):
    ret={}
    for c in data.columns:
        if c !=label:
            s=sorted(gini_func(train_data2,c,label).items(),key=lambda x:x[1])
#             print(s)
            if len(s)==2:
                temp1=s[0]
                temp2=s[1]
#                 print(temp1[0])
                if len(data[data[c]==temp1[0]][label].unique()) ==1:
                    print(c)
                    print(temp1)
                    ret[c]=temp1
                else :
                    ret[c]=temp2
            else:
#                 print(c)
                ret[c]=s[0]
    return ret


In [205]:
min_gini(train_data2,'Category')

{'Age': ('老年', 0.44),
 'Hourse': ('是', 0.26666666666666666),
 'Loan': ('一般', 0.32),
 'Work': ('是', 0.32)}

In [ ]:
[('Hourse', ('是', 0.26666666666666666)),
 ('Work', ('是', 0.32)),
 ('Loan', ('一般', 0.32)),
 ('Age', ('老年', 0.44))]

In [363]:
def create_dttree_cart(data,label):
    col=data.columns.values.tolist()
    s=sorted(min_gini(data,label).items(),key=lambda x:x[1][1])
#     s=min_gini(data,label)
    for i in range(len(s)-1):
        temp1=s[i]
        temp2=s[i+1]
        if temp1[1][1]==temp2[1][1]:
            if len(data[data[temp1[0]]==temp1[1][0]][label].unique())!=1:
                s[i]=temp2
                s[i+1]=temp1
    print(s)            


    gn,gvs=s[0] 
    gv=gvs[0]
    print(gn)
    print(gv)
    print(data)
    
    if data[data[gn]==gv].shape[0]>=1:
        ls=sorted(dict(data[data[gn]==gv][label].value_counts()).items(),
                                key=lambda x:x[1])
        if len(ls)>=2:
            l=ls[0][0]
            nl=ls[1][0]
        else:
            l=ls[0][0]

        if len(s)>1:
            col.remove(gn)
            temp=create_dttree_cart(data[data[gn]!=gv][col],label)

            return dttree(gn,gv,l,temp)
        else:
            nls=data[data[gn]!=gv][label].unique().tolist()
            nls.remove(l)
            nl=nls[0]
            return dttree(gn,gv,l,nl)
    else :
        return dttree(None,None,None,None)
                
    
    

In [364]:
# sorted(dict(train_data2['Category'].value_counts()).items(),key=lambda x:x[1])[-1][0]

In [365]:
create_dttree_cart(train_data2,'Category')

[('Hourse', ('是', 0.26666666666666666)), ('Work', ('是', 0.32)), ('Loan', ('一般', 0.32)), ('Age', ('老年', 0.44))]
Hourse
是
   Age Work Hourse Loan Category
0   青年    否      否   一般        否
1   青年    否      否    好        否
2   青年    是      否    好        是
3   青年    是      是   一般        是
4   青年    否      否   一般        否
5   中年    否      否   一般        否
6   中年    否      否    好        否
7   中年    是      是    好        是
8   中年    否      是  非常好        是
9   中年    否      是  非常好        是
10  老年    否      是  非常好        是
11  老年    否      是    好        是
12  老年    是      否    好        是
13  老年    是      否  非常好        是
14  老年    否      否   一般        否
Work
('否', 0.31999999999999995)
[('Work', ('否', 0.31999999999999995)), ('Loan', ('一般', 0.32)), ('Age', ('老年', 0.44))]
Work
否
   Age Work Loan Category
0   青年    否   一般        否
1   青年    否    好        否
2   青年    是    好        是
4   青年    否   一般        否
5   中年    否   一般        否
6   中年    否    好        否
12  老年    是    好        是
13  老年    是  非常好   

('Hourse', '是', '是', ('Work', '否', '否', (None, None, None, None)))

In [238]:
a=sorted(min_gini(train_data2,'Category').items(),key=lambda x:x[1][1])
a

[('Hourse', ('是', 0.26666666666666666)),
 ('Work', ('是', 0.32)),
 ('Loan', ('一般', 0.32)),
 ('Age', ('老年', 0.44))]

In [223]:
for k,v in min_gini(train_data2,'Category').items():
    print(v[1])


0.44
0.32
0.26666666666666666
0.32


In [215]:
create_dttree_cart(train_data2,'Category')

{'Age': ('老年', 0.44), 'Work': ('是', 0.32), 'Hourse': ('是', 0.26666666666666666), 'Loan': ('一般', 0.32)}


{'Age': ('老年', 0.44),
 'Hourse': ('是', 0.26666666666666666),
 'Loan': ('一般', 0.32),
 'Work': ('是', 0.32)}